In [6]:
import pandas as pd
import tensorflow as tf 
import matplotlib.pyplot as plt 
import numpy as np 

2025-06-04 18:01:33.354600: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Matplotlib is building the font cache; this may take a moment.


In [14]:
data = pd.read_csv("../data/icml_face_data.csv")
data.head()

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [15]:
# train_data = data[data["Usage"] == "Training"]
# test_data = data[data["Usage"] == "Testing"]

# train_data.head()

print(data.columns)

Index(['emotion', ' Usage', ' pixels'], dtype='object')


Given that the column names contain spaces at random intervals, we perform some data cleaning tasks. 

In [16]:
data_columns = data.columns

for string in data_columns: 
    data = data.rename(columns = {string: string.strip()})

print(data.columns)

Index(['emotion', 'Usage', 'pixels'], dtype='object')


Now lets seperate the training and testing data. Lets first explore what Values the Usage column takes. 

In [19]:
data["Usage"].value_counts()

Usage
Training       28709
PublicTest      3589
PrivateTest     3589
Name: count, dtype: int64

So we will use the Public Test and Private Test and group them together in the testing data. 

In [20]:
train_data = data[data["Usage"] == "Training"]
test_data = data[data["Usage"] != "Training"]

train_data.head()

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [21]:
test_data.head()

,emotion,Usage,pixels
28709,0,PublicTest,254 254 254 254 254 249 255 160 2 58 53 70 77 ...
28710,1,PublicTest,156 184 198 202 204 207 210 212 213 214 215 21...
28711,4,PublicTest,69 118 61 60 96 121 103 87 103 88 70 90 115 12...
28712,6,PublicTest,205 203 236 157 83 158 120 116 94 86 155 180 2...
28713,3,PublicTest,87 79 74 66 74 96 77 80 80 84 83 89 102 91 84 ...


Given that we've now segregated our data, there is no reason to keep the usage column and so we simply drop it. 

In [23]:
train_data.drop(columns=["Usage"], inplace = True)
train_data.head()

/var/folders/9k/th5b0bj95938tdg9_4l923tc0000gn/T/ipykernel_64813/2287182761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.drop(columns=["Usage"], inplace = True)


,emotion,pixels
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [24]:
test_data.drop(columns=["Usage"], inplace = True)
test_data.head()

/var/folders/9k/th5b0bj95938tdg9_4l923tc0000gn/T/ipykernel_64813/3513069122.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(columns=["Usage"], inplace = True)


,emotion,pixels
28709,0,254 254 254 254 254 249 255 160 2 58 53 70 77 ...
28710,1,156 184 198 202 204 207 210 212 213 214 215 21...
28711,4,69 118 61 60 96 121 103 87 103 88 70 90 115 12...
28712,6,205 203 236 157 83 158 120 116 94 86 155 180 2...
28713,3,87 79 74 66 74 96 77 80 80 84 83 89 102 91 84 ...


Let us now analyse the type of data present in our Dataset to identify the most optimal classification algorithm and model training strategy. 